# Import 

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy

In [2]:
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initiate BratDirReader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read eHOST annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


# Define sampling functions

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

### Define CRF Wrapper

In [10]:
from CRFWrapper import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel
from ALSampler import SamplingSimulator, ModelSamplingSimulator, RandomSamplingSimulator

In [11]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Route', 'Duration', 'ADE', 'Frequency', 'Drug', 'Form', 'Strength', 'Reason', 'Dosage'}


In [12]:
crf_model=CRFModel(anno_types=annos)
r_simulator=RandomSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)

In [13]:
scores=r_simulator.simulate_rounds(boostrap_times=200)

2024-05-20 15:07:43.338 | INFO     | ALSampler:simulate_rounds:53 - simulate round 0.
2024-05-20 15:07:43.339 | DEBUG    | ALSampler:keep_sample:39 - The first round sampling will be random
2024-05-20 15:07:43.341 | INFO     | ALSampler:keep_sample:43 - current sampled: 30, remaining: 273
2024-05-20 15:07:47.028 | DEBUG    | CRFWrapper:fit:266 - Reset and train CRF model...
2024-05-20 15:08:43.217 | DEBUG    | CRFWrapper:fit:274 - Training complete.
2024-05-20 15:08:43.312 | DEBUG    | CRFWrapper:bootstrap_eval:299 - Predicting eval docs...
2024-05-20 15:09:20.092 | DEBUG    | CRFWrapper:bootstrap_eval:303 - Calculate scores from bootstrapping 200 times
2024-05-20 15:13:42.822 | DEBUG    | CRFWrapper:bootstrap_eval:313 - complete
2024-05-20 15:13:42.968 | DEBUG    | ALSampler:simulate_rounds:64 - {'precision': 0.9436780861622498, 'recall': 0.7790147749120776, 'f1': 0.8534724728682227}
2024-05-20 15:13:42.970 | INFO     | ALSampler:simulate_rounds:53 - simulate round 1.
2024-05-20 15:13

In [14]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [15]:
pd.options.display.float_format='{:,.5f}'.format

In [16]:
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.94368,0.93985,0.94710,0.77901,0.77207,0.78561,0.85347,0.84886,0.85785
1,0.94338,0.94028,0.94694,0.80879,0.80277,0.81530,0.87091,0.86682,0.87472
2,0.94310,0.93972,0.94635,0.82460,0.81890,0.83017,0.87988,0.87605,0.88364
3,0.94341,0.94031,0.94671,0.83337,0.82774,0.83865,0.88498,0.88099,0.88895
4,0.94353,0.94007,0.94692,0.84270,0.83770,0.84748,0.89027,0.88595,0.89391
5,0.94567,0.94242,0.94887,0.84481,0.83868,0.84999,0.89240,0.88843,0.89602
6,0.94601,0.94241,0.94902,0.84983,0.84407,0.85503,0.89534,0.89139,0.89877
7,0.94463,0.94081,0.94792,0.85420,0.84875,0.85912,0.89714,0.89283,0.90059
8,0.94566,0.94263,0.94877,0.85807,0.85292,0.86348,0.89973,0.89612,0.90357
9,0.94592,0.94269,0.94898,0.85964,0.85514,0.86487,0.90072,0.89721,0.90464


## Bootstrap 10 runs

In [17]:
boostrap_runs=3
total_round=10

In [18]:
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [19]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [20]:
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos)
    m_simulator=RandomSamplingSimulator(train_docs, total_round=total_round, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)
    scores=m_simulator.simulate_rounds()
    all_scores.append(scores) 

2024-05-20 16:54:15.257 | INFO     | __main__:<module>:3 - start run 0.
2024-05-20 16:54:15.259 | INFO     | ALSampler:simulate_rounds:53 - simulate round 0.
2024-05-20 16:54:15.261 | INFO     | ALSampler:keep_sample:43 - current sampled: 30, remaining: 273
2024-05-20 17:00:02.253 | INFO     | ALSampler:simulate_rounds:53 - simulate round 1.
2024-05-20 17:00:02.256 | INFO     | ALSampler:keep_sample:43 - current sampled: 60, remaining: 243
2024-05-20 17:06:46.626 | INFO     | ALSampler:simulate_rounds:53 - simulate round 2.
2024-05-20 17:06:46.630 | INFO     | ALSampler:keep_sample:43 - current sampled: 90, remaining: 213
2024-05-20 17:14:35.801 | INFO     | ALSampler:simulate_rounds:53 - simulate round 3.
2024-05-20 17:14:35.805 | INFO     | ALSampler:keep_sample:43 - current sampled: 120, remaining: 183
2024-05-20 17:23:37.081 | INFO     | ALSampler:simulate_rounds:53 - simulate round 4.
2024-05-20 17:23:37.084 | INFO     | ALSampler:keep_sample:43 - current sampled: 150, remaining: 

In [25]:
joblib.dump(all_scores, r'..\data\ner_rand_scores.joblib')

['..\\data\\ner_rand_scores.joblib']

In [27]:
pwd

'T:\\VINCI_NLPETL\\workspace\\jianlins\\PycharmProjects\\vectorsearch\\notebooks'

In [20]:
len(all_scores[0])

10

In [8]:
import random

In [39]:
random.seed(5)
[random.randint(0,10) for _ in range(20)]

[9, 4, 5, 10, 8, 0, 7, 3, 10, 0, 2, 1, 5, 7, 3, 6, 8, 1, 9, 3]